In [1]:
import os
import socket
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import BooleanType, IntegerType, LongType, StringType, ArrayType, FloatType, StructType, StructField
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from pyspark import StorageLevel
from jinja2 import Environment, FileSystemLoader

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, length, when, col
from emoji import get_emoji_regexp, unicode_codes
import re

In [2]:
# setting constants
APP_NAME = "YOUR_APP_NAME"
NORMALIZED_APP_NAME = APP_NAME.replace('/', '_').replace(':', '_')

APPS_TMP_DIR = os.path.join(os.getcwd(), "tmp")
APPS_CONF_DIR = os.path.join(os.getcwd(), "conf")
APPS_LOGS_DIR = os.path.join(os.getcwd(), "logs")
LOG4J_PROP_FILE = os.path.join(APPS_CONF_DIR, "pyspark-log4j-{}.properties".format(NORMALIZED_APP_NAME))
LOG_FILE = os.path.join(APPS_LOGS_DIR, 'pyspark-{}.log'.format(NORMALIZED_APP_NAME))
EXTRA_JAVA_OPTIONS = "-Dlog4j.configuration=file://{} -Dspark.hadoop.dfs.replication=1 -Dhttps.protocols=TLSv1.0,TLSv1.1,TLSv1.2,TLSv1.3"\
    .format(LOG4J_PROP_FILE)

LOCAL_IP = socket.gethostbyname(socket.gethostname())

# preparing configuration files from templates
for directory in [APPS_CONF_DIR, APPS_LOGS_DIR, APPS_TMP_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

env = Environment(loader=FileSystemLoader('/opt'))
template = env.get_template("pyspark_log4j.properties.template")
template\
    .stream(logfile=LOG_FILE)\
    .dump(LOG4J_PROP_FILE)

# run spark
spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master("k8s://https://10.32.7.103:6443")\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.ui.port", "4040")\
    .config("spark.kubernetes.memoryOverheadFactor", "0.6")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "5")\
    .config("spark.executor.cores", '4')\
    .config("spark.executor.memory", "5g")\
    .config("spark.memory.fraction", "0.6")\
    .config("spark.memory.storageFraction", "0.5")\
    .config("spark.sql.autoBroadcastJoinThreshold", "-1")\
    .config("spark.driver.extraJavaOptions", EXTRA_JAVA_OPTIONS)\
    .config("spark.kubernetes.namespace", "gkulagin-307618")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.container.image.pullPolicy", "Always")\
    .config("spark.kubernetes.container.image", "node03.st:5000/spark-executor:gkulagin-307618")\
    .config("spark.kubernetes.executor.deleteOnTermination", "true")\
    .config("spark.local.dir", "/tmp/spark")\
    .getOrCreate()

In [3]:
import os
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession

from pyspark.sql.types import BooleanType, IntegerType, LongType, StringType, ArrayType, FloatType, StructType, StructField
import pyspark.sql.functions as F
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from jinja2 import Environment, FileSystemLoader

#os.environ["PYSPARK_PYTHON"] = "/home/nikolay/.conda/envs/default/bin/python"
os.environ["PYSPARK_PYTHON"]="/opt/conda/bin/python3.8"
# setting constants
SPARK_ADDRESS = "local[4]"

APP_NAME = "practice_3"
NORMALIZED_APP_NAME = APP_NAME.replace('/', '_').replace(':', '_')

APPS_TMP_DIR = os.path.join(os.getcwd(), "tmp")
APPS_CONF_DIR = os.path.join(os.getcwd(), "conf")
APPS_LOGS_DIR = os.path.join(os.getcwd(), "logs")
LOG4J_PROP_FILE = os.path.join(APPS_CONF_DIR, "pyspark-log4j-{}.properties".format(NORMALIZED_APP_NAME))
LOG_FILE = os.path.join(APPS_LOGS_DIR, 'pyspark-{}.log'.format(NORMALIZED_APP_NAME))
EXTRA_JAVA_OPTIONS = "-Dlog4j.configuration=file://{} -Dspark.hadoop.dfs.replication=1"\
    .format(LOG4J_PROP_FILE)

# preparing configuration files from templates
for directory in [APPS_CONF_DIR, APPS_LOGS_DIR, APPS_TMP_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

env = Environment(loader=FileSystemLoader('/opt'))
template = env.get_template("pyspark_log4j.properties.template")
template\
    .stream(logfile=LOG_FILE)\
    .dump(LOG4J_PROP_FILE)

# run spark
spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master(SPARK_ADDRESS)\
    .config('spark.ui.port', "4040")\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.6")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.extraJavaOptions", EXTRA_JAVA_OPTIONS)\
    .config("spark.executor.memory", "6g")\
    .getOrCreate()

# printing important urls and pathes
print("Web UI: {}".format(spark.sparkContext.uiWebUrl))
print("\nlog4j file: {}".format(LOG4J_PROP_FILE))
print("\ndriver log file: {}".format(LOG_FILE))

Web UI: http://10.128.1.33:4040

log4j file: /home/jovyan/nfs-home/conf/pyspark-log4j-practice_3.properties

driver log file: /home/jovyan/nfs-home/logs/pyspark-practice_3.log


In [4]:
# spark.stop()

# Read all data

In [5]:
posts_df = spark.read.json("/shared/bigdata20/posts_api.json")

In [6]:
posts_likes_df = spark.read.parquet("/shared/bigdata20/posts_likes.parquet")
followers_df = spark.read.parquet("/shared/bigdata20/followers.parquet")
followers_posts_df = spark.read.json("/shared/bigdata20/followers_posts_api_final.json")
followers_posts_likes_df = spark.read.parquet("/shared/bigdata20/followers_posts_likes.parquet")

## Task 1 
Find the top 20 posts in the group: (a) by likes; (b) by comments; (c) by reposts. 

In [7]:
posts_df.printSchema()

root
 |-- attachments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- album: struct (nullable = true)
 |    |    |    |-- created: long (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |-- size: long (nullable = true)
 |    |    |    |-- thumb: struct (nullable = true)
 |    |    |    |    |-- access_key: string (nullable = true)
 |    |    |    |    |-- album_id: long (nullable = true)
 |    |    |    |    |-- date: long (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |    |-- sizes: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |    |-- type: string (nullable = true)
 |   

In [8]:
#(a) by likes
posts_df.select('id', col('likes.count').name('likes count'))\
        .orderBy("likes.count", ascending=False)\
        .show(20)

+-----+-----------+
|   id|likes count|
+-----+-----------+
|32022|       1637|
|35068|       1629|
|17492|       1516|
|18526|       1026|
|19552|        955|
|41468|        952|
|19419|        868|
|29046|        824|
|32546|        786|
|24085|        765|
|40180|        759|
|33658|        708|
|13532|        633|
|40842|        631|
|35117|        588|
|17014|        581|
|19583|        553|
|19809|        552|
|27455|        550|
|11999|        549|
+-----+-----------+
only showing top 20 rows



In [9]:
#(b) by comments
posts_df.select('id', col('comments.count').name('comments count'))\
        .orderBy("comments.count", ascending=False)\
        .show(20)

+-----+--------------+
|   id|comments count|
+-----+--------------+
|24085|           850|
|22540|           250|
|27722|           192|
| 8285|           148|
|26860|           113|
|13571|           107|
|39294|           104|
|36680|            96|
|26006|            92|
|41739|            92|
|12426|            91|
|21499|            88|
|39407|            83|
|39163|            83|
|11267|            81|
|31548|            80|
|11158|            70|
|39082|            67|
|14602|            61|
|12687|            61|
+-----+--------------+
only showing top 20 rows



In [10]:
#(c) by reposts
posts_df.select('id', col('reposts.count').name('reposts count'))\
        .orderBy("reposts.count", ascending=False)\
        .show(20)

+-----+-------------+
|   id|reposts count|
+-----+-------------+
|17492|          334|
|19552|          246|
|32022|          210|
|11842|          129|
|19419|          126|
|13532|          110|
|17014|          105|
|35068|          101|
|41266|           92|
|12593|           90|
|29046|           87|
|11999|           85|
|41468|           85|
|19809|           84|
|17167|           81|
|10833|           78|
|18543|           77|
|16596|           76|
|18156|           74|
|37262|           71|
+-----+-------------+
only showing top 20 rows



## Task 2 
Find the top 20 users by (a) likes and (b) reposts they have made (to trace reposts use "copy_history" field) 

In [11]:
posts_likes_df.printSchema()

root
 |-- itemType: string (nullable = true)
 |-- ownerId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- likerId: integer (nullable = true)



In [12]:
#top 20 users by likes they have made
posts_likes_df.groupby('likerId')\
              .agg(F.count('itemId').name('likes_count'))\
              .orderBy('likes_count', ascending=False)\
              .select(col('likerId').name('user_id'), 'likes_count')\
              .show(20)

+---------+-----------+
|  user_id|likes_count|
+---------+-----------+
|  2070090|       4801|
|  2397858|       2055|
|  1475301|       1829|
|    18239|       1569|
|   546612|       1245|
|     6371|        907|
|  1841959|        746|
| 78440957|        709|
|   120248|        699|
| 40981497|        611|
|    22158|        553|
|207628162|        548|
|329377723|        504|
| 76071304|        474|
| 14805173|        440|
|   317799|        385|
| 56355640|        375|
| 52042971|        338|
|  7437271|        336|
|136506644|        335|
+---------+-----------+
only showing top 20 rows



In [13]:
followers_posts_df.printSchema()

root
 |-- attachments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- album: struct (nullable = true)
 |    |    |    |-- created: long (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |-- size: long (nullable = true)
 |    |    |    |-- thumb: struct (nullable = true)
 |    |    |    |    |-- access_key: string (nullable = true)
 |    |    |    |    |-- album_id: long (nullable = true)
 |    |    |    |    |-- date: long (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- lat: double (nullable = true)
 |    |    |    |    |-- long: double (nullable = true)
 |    |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |    |-- sizes: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    

In [14]:
#top 20 users by reposts they have made
followers_posts_df.select('owner_id', col("copy_history.id").getItem(0).name('post_id'))\
        .na.drop()\
        .groupby('owner_id')\
        .agg(F.count('post_id').name('posts_num'))\
        .orderBy('posts_num', ascending=False)\
        .show()

+---------+---------+
| owner_id|posts_num|
+---------+---------+
|  2547211|    37742|
|357231922|    23349|
|168543860|    18429|
| 25646344|    11122|
|176861294|     9022|
|524656784|     7242|
|    29840|     7164|
|143207077|     7161|
|141687240|     6804|
|459339006|     6741|
|514384760|     6570|
|483715951|     6052|
|445159771|     5808|
|451211328|     5646|
|426396104|     5533|
|  8325325|     5532|
|452280411|     5458|
|464220898|     5318|
|440454268|     5304|
|461319529|     5240|
+---------+---------+
only showing top 20 rows



## Task 3 
get reposts of the original posts of the itmo group (posts.json) from user posts (the result should be similar to (group_post_id, Array (user_post_ids)))

In [15]:
followers_posts_df.select(
                        col('id').name('user_post_id'),
                        col("copy_history.id").getItem(0).name("post_id"),
                        col("copy_history.owner_id").getItem(0).name("owner_id")
                        )\
                  .join(
                        posts_df.select(col('id').name('post_id'), "owner_id"),
                        ['owner_id', 'post_id']
                        )\
                  .groupBy('post_id')\
                  .agg(F.collect_list('user_post_id').name('user_post_ids'))\
                  .withColumn('reposts_num', F.size('user_post_ids'))\
                  .orderBy('reposts_num', ascending=False)\
                  .show()
                                       

+-------+--------------------+-----------+
|post_id|       user_post_ids|reposts_num|
+-------+--------------------+-----------+
|  41266|[1150, 235, 452, ...|         30|
|  41468|[15321, 400, 2561...|         25|
|  42482|[4205, 713, 1542,...|         10|
|  40090|[463, 1325, 32, 3...|          9|
|  39259|[10810, 1205, 822...|          8|
|  38740|[185, 8186, 1133,...|          8|
|  41207|[10610, 1288, 296...|          6|
|  41721|[10896, 274, 4865...|          6|
|  41546|[939, 1161, 11609...|          6|
|  41506|[12823, 135, 397,...|          5|
|  38963|[3720, 6403, 814,...|          5|
|  39682|[384, 159, 3262, ...|          5|
|  42730|[7129, 5561, 6840...|          4|
|  38915|[18054, 4487, 118...|          4|
|  41108|[10483, 182, 1055...|          4|
|  41708|[5112, 271, 10107...|          4|
|  39515|[2321, 9218, 443,...|          4|
|  39294|[4516, 2319, 4718...|          4|
|  39686|[3370, 4741, 182,...|          4|
|  40011| [4770, 19267, 2559]|          3|
+-------+--

## Task 4 
find emoticons in posts (negative, positive, neutral), calculate their overall count,  frequency (number of posts they can be found in) and average count per post. Print top 10 most popular emoticons, print top 5 emoticons which have the greatest difference between their overall count and frequency, print top 5 emoticons with average count per post.


In [22]:
!pip install --user --trusted-host pypi-registry.supplementary-services.svc.cluster.local --index http://pypi-registry.supplementary-services.svc.cluster.local:8080/ emoji

Looking in indexes: http://pypi-registry.supplementary-services.svc.cluster.local:8080/


In [23]:
import sys
!{sys.executable} -m pip install --user --trusted-host pypi-registry.supplementary-services.svc.cluster.local --index http://pypi-registry.supplementary-services.svc.cluster.local:8080/ emoji


Looking in indexes: http://pypi-registry.supplementary-services.svc.cluster.local:8080/


In [16]:
from emoji import get_emoji_regexp, unicode_codes

In [17]:
@udf(returnType=ArrayType(StringType()))
def get_emoji_udf(text):
    all_matches = get_emoji_regexp().finditer(text)
    emoji = []
    for match in all_matches:
        emoji.append(match.group())
    
    return emoji

In [18]:
posts_df.printSchema()

root
 |-- attachments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- album: struct (nullable = true)
 |    |    |    |-- created: long (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |-- size: long (nullable = true)
 |    |    |    |-- thumb: struct (nullable = true)
 |    |    |    |    |-- access_key: string (nullable = true)
 |    |    |    |    |-- album_id: long (nullable = true)
 |    |    |    |    |-- date: long (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |    |-- sizes: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |    |-- type: string (nullable = true)
 |   

In [19]:
emoji_df = posts_df.where("text <> ''").select('id', get_emoji_udf(col("text")).name('emoji'))\
        .where(F.size('emoji') > 0)\
        .withColumn('emoji', F.explode("emoji"))\
        .groupBy('emoji')\
        .agg(F.count('id').name('all_count'), F.countDistinct('id').name('posts_count'))\
        .withColumn('avg_per_post', col('all_count') / col('posts_count'))\
        .withColumn('difference', col('all_count') - col('posts_count'))\
        .orderBy(F.desc('all_count'))


emoji_df.limit(5)\
        .toPandas()

,emoji,all_count,posts_count,avg_per_post,difference
0,🔥,76,61,1.245902,15
1,⚡,68,45,1.511111,23
2,📍,63,39,1.615385,24
3,🚀,50,47,1.063830,3
4,❤,47,47,1.000000,0


Print top 10 most popular emoticons

In [20]:
emoji_df.limit(10)\
        .toPandas()

,emoji,all_count,posts_count,avg_per_post,difference
0,🔥,76,61,1.245902,15
1,⚡,68,45,1.511111,23
2,📍,63,39,1.615385,24
3,🚀,50,47,1.063830,3
4,❤,47,47,1.000000,0
5,❗,45,20,2.250000,25
6,✔,40,9,4.444444,31
7,🔵,40,17,2.352941,23
8,⬇,40,16,2.500000,24
9,💙,38,38,1.000000,0


Print top 5 emoticons which have the greatest difference between their overall count and frequency

In [21]:
emoji_df.orderBy(F.desc('difference'))\
        .limit(5)\
        .toPandas()

,emoji,all_count,posts_count,avg_per_post,difference
0,✔,40,9,4.444444,31
1,❗,45,20,2.250000,25
2,⬇,40,16,2.500000,24
3,📍,63,39,1.615385,24
4,🔹,30,7,4.285714,23


Print top 5 emoticons with average count per post

In [22]:
emoji_df.orderBy(F.desc('avg_per_post'))\
        .limit(5)\
        .toPandas()

,emoji,all_count,posts_count,avg_per_post,difference
0,✔,40,9,4.444444,31
1,🔹,30,7,4.285714,23
2,▪,21,5,4.200000,16
3,✒,7,2,3.500000,5
4,📷,3,1,3.000000,2


## Task 5 
Probable “fans”. Find for each user the top 10 other users whose posts this user likes. 

In [23]:
followers_posts_likes_df.printSchema()

root
 |-- itemType: string (nullable = true)
 |-- ownerId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- likerId: integer (nullable = true)



In [24]:
from pyspark.sql.window import Window

In [25]:
followers_posts_likes_df.select(col('ownerId').name('user_id'), 'itemId', 'likerId')\
                        .groupBy('likerId', 'user_id')\
                        .agg(F.count('itemId').name("likes_count"))\
                        .withColumn("top", F.row_number().over(Window.partitionBy("likerId").orderBy(F.desc("likes_count"))))\
                        .where('top <= 10')\
                        .orderBy('likerId', F.desc("likes_count"))\
                        .show()

+-------+--------+-----------+---+
|likerId| user_id|likes_count|top|
+-------+--------+-----------+---+
|      9|  654356|          4|  1|
|     14| 1986125|          2|  1|
|     14|    3420|          1|  4|
|     14| 3680017|          1|  2|
|     14|   35524|          1|  3|
|     15| 1227412|          2|  2|
|     15|  473831|          2|  1|
|     17| 1986125|         10|  1|
|     17|    3420|          3|  2|
|     17|   88060|          2|  3|
|     34|   50601|          1|  1|
|     34| 1986125|          1|  2|
|     63|   73115|          1|  1|
|     99| 1550591|          1|  1|
|    122|   55983|          1|  1|
|    143|     637|          3|  1|
|    146|19515730|          3|  1|
|    146|  106610|          2|  2|
|    149| 1986125|          1|  1|
|    154| 1601367|          1|  1|
+-------+--------+-----------+---+
only showing top 20 rows



## Task 6
Probable friends. If two users like each other posts they may be friends. Find pairs of users where both users are top likers of each other.

In [26]:
posts_likes_df.printSchema()

root
 |-- itemType: string (nullable = true)
 |-- ownerId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- likerId: integer (nullable = true)



In [27]:
user_df_1 = followers_posts_likes_df\
    .select(col("ownerId").name("user_1"), col("likerId").alias("user_2"), col("itemId"))

user_df_2 = followers_posts_likes_df\
    .select(col("ownerId").name("user_2"), col("likerId").alias("user_1"))

user_df_3 = user_df_1.join(user_df_2, ["user_1", "user_2"]).where("user_1 != user_2")\
    .groupBy("user_1", "user_2").agg(F.count("itemId").alias("cnt_posts"))\
    .orderBy(F.desc("cnt_posts"))

In [28]:
user_df_3.withColumn('first_user', F.least('user_1', 'user_2'))\
         .withColumn('second_user', F.greatest('user_1', 'user_2'))\
         .groupBy('first_user', 'second_user')\
         .agg(F.min('cnt_posts').name('cnt_posts'))\
         .orderBy(F.desc('cnt_posts'))\
         .show()

+----------+-----------+---------+
|first_user|second_user|cnt_posts|
+----------+-----------+---------+
|  13675440|  183535934|    16100|
|   1475301|    5633955|     1972|
| 207134315|  208946862|     1612|
| 145105762|  267301242|     1456|
|    108408|    7697818|      880|
|    135451|   18737802|      810|
| 209077977|  272076217|      720|
| 155963006|  162366815|      684|
|   2547211|    4448812|      608|
|  19261491|  229861638|      598|
|  53368685|  322831238|      572|
|  52612744|   53720099|      544|
|  66022003|   95356919|      506|
|  83892412|  115252127|      390|
|  47122493|   63363182|      376|
|  27102997|   74634237|      330|
|  44770563|  103229751|      330|
| 101767883|  188548515|      312|
|   1475301|    4068532|      300|
|  34892097|   59949877|      299|
+----------+-----------+---------+
only showing top 20 rows

